# Clickbait Spoiler Generation using GPT-3

In [ ]:
# This is necessary to fix the imports
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../src')))

Preparing the data using a custom function and validating it using the openai preparation tool

In [ ]:
import models.gpt3 as gpt3
import utils.openai

from prepare_data_openai import OPENAI_MODEL

In [ ]:
MODEL_ID = "ada:ft-personal-2023-05-04-01-44-28"
OPENAI_MAX_EVAL_LEN = 20

In [ ]:
validation = gpt3.read_data("../data/parsed/openai/validation.jsonl")

In [ ]:
utils.openai.estimate_costs_fine_tune_usage("../data/parsed/openai/validation.jsonl", OPENAI_MODEL)

In [ ]:
val_predictions = gpt3.predict(validation["prompts"], MODEL_ID)

In [ ]:
gpt3.write_results(validation["prompts"], validation["completions"], val_predictions, "../data/results/validation.jsonl")

In [ ]:
import evaluate
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

In [ ]:
meteor_results = meteor.compute(predictions=val_predictions[0], references=[validation["completions"][0]])
bleu_results = bleu.compute(predictions=val_predictions[0], references=[validation["completions"][0]])
bertscore_results = bertscore.compute(predictions=val_predictions[0], references=[validation["completions"][0]], lang="en")

print(f"Meteor: {meteor_results['meteor']}\nBLEU-4: {bleu_results['bleu']}\nBERTscore Mean F1: {sum(bertscore_results['f1'])/len(bertscore_results['f1'])}")

In [ ]:
test = gpt3.read_data("../data/parsed/openai/test.jsonl")

In [ ]:
utils.openai.estimate_costs_fine_tune_usage("../data/parsed/openai/test.jsonl", OPENAI_MODEL)

In [ ]:
test_predictions = gpt3.predict(test["prompts"], MODEL_ID)

In [ ]:
gpt3.write_results(test["prompts"], test["completions"], test_predictions, "../data/results/test.jsonl")

In [ ]:
meteor_results = meteor.compute(predictions=test_predictions, references= test["completions"])
bleu_results = bleu.compute(predictions=test_predictions, references= test["completions"])
bertscore_results = bertscore.compute(predictions=test_predictions, references= test["completions"], lang="en")

print(f"Meteor: {meteor_results['meteor']}\nBLEU-4: {bleu_results['bleu']}\nBERTscore Mean F1: {sum(bertscore_results['f1'])/len(bertscore_results['f1'])}")